In [2]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt

In [3]:
words = open('names.txt', 'r').read().splitlines()
len(words)

32033

In [4]:
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
itos

{1: 'a',
 2: 'b',
 3: 'c',
 4: 'd',
 5: 'e',
 6: 'f',
 7: 'g',
 8: 'h',
 9: 'i',
 10: 'j',
 11: 'k',
 12: 'l',
 13: 'm',
 14: 'n',
 15: 'o',
 16: 'p',
 17: 'q',
 18: 'r',
 19: 's',
 20: 't',
 21: 'u',
 22: 'v',
 23: 'w',
 24: 'x',
 25: 'y',
 26: 'z',
 0: '.'}

In [5]:
block_size = 3 # context length: how many chars to take to predict the next
X, Y = [], []
for w in words[:5]:
  print(w)
  context = [0] * block_size
  for ch in w + '.':
    ix = stoi[ch]
    X.append(context)
    Y.append(ix)
    print(''.join(itos[i] for i in context), '---->', itos[ix])
    context = context[1:] + [ix] # crop and append

X =  torch.tensor(X)
Y = torch.tensor(Y)
X.shape, X.dtype

emma
... ----> e
..e ----> m
.em ----> m
emm ----> a
mma ----> .
olivia
... ----> o
..o ----> l
.ol ----> i
oli ----> v
liv ----> i
ivi ----> a
via ----> .
ava
... ----> a
..a ----> v
.av ----> a
ava ----> .
isabella
... ----> i
..i ----> s
.is ----> a
isa ----> b
sab ----> e
abe ----> l
bel ----> l
ell ----> a
lla ----> .
sophia
... ----> s
..s ----> o
.so ----> p
sop ----> h
oph ----> i
phi ----> a
hia ----> .


(torch.Size([32, 3]), torch.int64)

In [6]:
C =  torch.randn((27, 2)) # each 27 chars will have 2 dim space
C

tensor([[-0.5187, -0.6635],
        [-1.2788, -0.2109],
        [-1.3987, -1.0864],
        [ 0.3764, -0.2688],
        [ 0.1555,  0.9545],
        [ 0.9076, -1.2435],
        [ 0.6890,  0.0132],
        [-0.8300,  0.2763],
        [ 0.3329,  1.6711],
        [-0.0079, -0.6316],
        [-0.0263, -0.0091],
        [ 0.3656, -0.6223],
        [-1.5174,  0.9439],
        [-1.1339,  1.3864],
        [-0.7596,  2.5453],
        [-0.7795,  1.7824],
        [-0.1771,  1.4251],
        [-1.1884, -0.2153],
        [-0.0989, -1.0518],
        [-1.4279, -0.7732],
        [-0.2226, -0.2716],
        [ 0.8863,  2.0756],
        [ 0.4554,  1.6259],
        [-0.9193, -0.1384],
        [-1.4129, -0.8895],
        [-1.0657,  0.4222],
        [-1.7512,  0.1753]])

In [7]:
F.one_hot(torch.tensor(5), num_classes=27).float() @ C
# is equivalent to
C[5]

tensor([ 0.9076, -1.2435])

In [16]:
C.shape #27, 2
X.shape #32, 3
C[torch.tensor([5,5])] # you can index using tensor

tensor([[ 0.9076, -1.2435],
        [ 0.9076, -1.2435]])

In [18]:
# you can index using multi dim
# X[something, something] is integer between 0 to 26. (something, something) is used in place of row index of C(27, 2), forming (something, 3, 2). think of this like matrix of row=(something,3), col=2. (something, 3) designates 27 different characters. 2 is size of feature vector.
C[X].shape

torch.Size([32, 3, 2])

In [19]:
X[13,2]

tensor(1)

In [20]:
C[X][13, 2]

tensor([-1.2788, -0.2109])

In [21]:
C[1]

tensor([-1.2788, -0.2109])

In [24]:
emb = C[X]
W1 = torch.randn((6, 100)) # 6 = 3 (blocksize) * 2 (embedding dimention) # num of neuron is up to us, 100
b1 = torch.randn(100)
# want to do emb @ W1 + b1, but then emb has to be (32, 6)

In [29]:
emb[:, 0, :].shape # take the first(0) character for all 32 blocks and corresponding feature vector.

torch.Size([32, 2])

In [33]:
torch.cat([emb[:, 0, :], emb[:, 1, :], emb[:, 2, :]], 1).shape # concat by the second (1) dimention.
# the first 2 of 6 col is the feature vector for the first char in a block.
# the second 2 of 6 is the feature for the second in a block, and so is for third.

torch.Size([32, 6])

In [41]:
torch.unbind(torch.tensor([[1, 2, 3],
                           [4, 5, 6],
                           [7, 8, 9]]), 1) # unbind splits tensor by dimention 1 (col?)

(tensor([1, 4, 7]), tensor([2, 5, 8]), tensor([3, 6, 9]))

In [68]:
# emb:
# 
# [[][]] [[][]] [[][]]
# [[][]] [[][]] [[][]]
# ...    ...    ...
len(torch.unbind(emb, 1)) #3 #split tensor by col. form array of length 3, each element is (32, 2)
torch.tensor(torch.unbind(emb, 1)[0]).shape #(32, 2)
torch.cat(torch.unbind(emb, 1), 1).shape # (32, 6) same as the cell above*2

/tmp/ipykernel_723480/1562459235.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(torch.unbind(emb, 1)[0]).shape #(32, 2)


torch.Size([32, 6])

In [77]:
emb.view(32, 6) # much efficient

tensor([-0.5187, -0.6635])

In [83]:
h = emb.view(-1, 6) @ W1 + b1
h = torch.tanh(h)
h.shape

torch.Size([32, 100])